In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from pandas.testing import assert_frame_equal
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [2]:
dataset = pd.read_csv('data/train_v1.csv')
testset = pd.read_csv('data/test_v3.csv')
print(dataset.shape)
print(testset.shape)
# print(dataset.dtypes)
# print(testset.dtypes)
testset = testset.drop(labels=['Unnamed: 0_x',"Unnamed: 0_y",'Unnamed: 0'],axis=1)
dataset = dataset.drop(labels=['Unnamed: 0'],axis=1)

totalset = pd.concat([dataset,testset])
lb = preprocessing.LabelEncoder()

# print(dataset['accessories_vectors'])
# totalset['make'] = lb.fit_transform(totalset['make'].values)
# totalset['model'] = lb.fit_transform(totalset['model'].values)
totalset['accessories_vectors'] = lb.fit_transform(totalset['accessories_vectors'].values)







(16727, 36)
(5000, 37)


In [3]:
# functions to fill NaN price
def get_mean_value(train,make,model,feature):
   
    mean_val = train.loc[(train.make == make) & (train.model == model)][feature].dropna().mean()
    
    if np.isnan(mean_val):
        mean_val = train.loc[(train.make == make)][feature].dropna().mean()
        if np.isnan(mean_val):
            mean_val = train[feature].dropna().mean()         
    return mean_val

def fill_nan(train,feature):
    for index, row in train.iterrows():
#     print(row)
        if np.isnan(row[feature]):
#             if np.isnan(row.model):
#                 model = fillMake['model'][index]
#             else:
#                 model = row.model    
            train[feature][index] = get_mean_value(train,row.make,row.model,feature)

In [4]:
# deal with make_model dataset
make_model = pd.read_csv('data/make_model.csv')
make_model = make_model.drop(labels=['index'],axis=1)

make_model['make'] = make_model['mod'].str.split(' ').str[0]
make_model['model'] = make_model['mod'].str.split(' ').str[1]

make_model = make_model.drop(labels=['mod'],axis=1)
make_model['is_new'].fillna(0, inplace=True)
make_model['is_new'].fillna(0, inplace=True)
make_model['is_new'].replace('Past',1,inplace=True) 

make_model.rename(columns={'price':'model_price'},inplace=True)
make_model['model_price'].replace(-1,np.NaN,inplace=True) 
fill_nan(make_model,'model_price')

print(make_model)

<ipython-input-3-a148c5a91eef>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[feature][index] = get_mean_value(train,row.make,row.model,feature)


     model_price  is_new           make     model
0       243888.0       0            bmw      320i
1        75000.0       0         toyota     hiace
2       170800.0       0  mercedes-benz    cla180
3       240888.0       1  mercedes-benz      e180
4       133099.0       0          honda     civic
..           ...     ...            ...       ...
741      96000.0       1           hino   fy1etkm
742      65999.0       1      chevrolet     spark
743      95888.0       1  mercedes-benz      a170
744      61999.0       1            kia  magentis
745     300000.0       0          volvo       v90

[746 rows x 4 columns]


In [5]:
# merge make_mode dataset with the original dataset
totalset = pd.merge(totalset, make_model, how='left', on=['make', 'model'])
totalset['make'] = lb.fit_transform(totalset['make'].values)
totalset['model'] = lb.fit_transform(totalset['model'].values)
print(totalset.columns)

Index(['listing_id', 'accessories_vectors', 'years of warranty',
       'better loan offer', 'well maintained', 'low fuel consumption',
       'reg_date', 'power', 'engine_cap', 'mileage', 'no_of_owners',
       'depreciation', 'coe', 'dereg_value', 'omv', 'arf',
       'type_of_vehicle_bus/mini bus', 'type_of_vehicle_hatchback',
       'type_of_vehicle_luxury sedan', 'type_of_vehicle_mid-sized sedan',
       'type_of_vehicle_mpv', 'type_of_vehicle_others',
       'type_of_vehicle_sports car', 'type_of_vehicle_stationwagon',
       'type_of_vehicle_suv', 'type_of_vehicle_truck', 'type_of_vehicle_van',
       'fuel_type_diesel', 'fuel_type_electric', 'fuel_type_petrol',
       'fuel_type_petrol-electric', 'transmission', 'make', 'model', 'price',
       'model_price', 'is_new'],
      dtype='object')


In [7]:
dataset = totalset.head(16727)
testset = totalset.tail(5000)

# print(testset.head(5))

testset = testset.drop(labels=['price'],axis=1)
print(dataset.shape)
print(testset.shape)

train, test = train_test_split(dataset, test_size=0.2, random_state=0)
# print(dataset['accessories_vectors'])

dataset.to_csv("train_v5.csv")
testset.to_csv("test_v5.csv")

(16727, 37)
(5000, 36)
